In [1]:
%pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=87930 sha256=e4d2be7ee11617825a75f731d5cc2fdecef6fd60e6214b4f73cac7b0cdff41f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-j3_ug15a/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_HARD

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.datasets import GroupLabeledDatasetWrapper

group_labeled_trainset = GroupLabeledDatasetWrapper(trainset, trainset.group_partition)

In [6]:
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

In [7]:
from torch.optim import SGD
from spuco.invariant_train import GroupDRO

group_dro = GroupDRO(
    model=model,
    num_epochs=1,
    trainset=group_labeled_trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_dro.train()

Epoch 0: 100%|██████████| 751/751 [00:16<00:00, 45.29batch/s, accuracy=75.0%, loss=0.107]   


In [8]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.02it/s]

Group (0, 0) Test Accuracy: 88.65248226950355


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:08,  2.77it/s]

Group (0, 1) Test Accuracy: 88.65248226950355


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:08,  2.50it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:09<00:00,  2.49it/s]

Group (4, 3) Test Accuracy: 82.07070707070707


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:10<00:00,  2.41it/s]

Group (4, 4) Test Accuracy: 85.60606060606061


{(0, 0): 88.65248226950355,
 (0, 1): 88.65248226950355,
 (0, 2): 87.70685579196217,
 (0, 3): 88.17966903073287,
 (0, 4): 87.47044917257683,
 (1, 0): 78.72860635696821,
 (1, 1): 77.75061124694376,
 (1, 2): 73.52941176470588,
 (1, 3): 78.18627450980392,
 (1, 4): 74.50980392156863,
 (2, 0): 74.13333333333334,
 (2, 1): 76.8,
 (2, 2): 83.2,
 (2, 3): 74.66666666666667,
 (2, 4): 70.32085561497327,
 (3, 0): 81.90954773869346,
 (3, 1): 81.360201511335,
 (3, 2): 80.85642317380352,
 (3, 3): 79.84886649874055,
 (3, 4): 78.84130982367758,
 (4, 0): 81.10831234256926,
 (4, 1): 78.84130982367758,
 (4, 2): 75.81863979848866,
 (4, 3): 82.07070707070707,
 (4, 4): 85.60606060606061}

In [9]:
evaluator.worst_group_accuracy

((2, 4), 70.32085561497327)

In [10]:
evaluator.average_accuracy

83.029447895627

In [11]:
evaluator.evaluate_spurious_task()

21.08